In [1]:
!pip install -q torch opencv-python pandas
!pip install -q kaggle
!pip install -q ultralytics matplotlib
!pip install -q huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00


In [2]:
!kaggle datasets download -d ayushspai/sportsmot -q
!unzip -q sportsmot.zip -d SportsMOT
!rm sportsmot.zip
# !mv /content/SportsMOT/sportsmot_publish/dataset/val/* /content/SportsMOT/sportsmot_publish/dataset/train > /dev/null 2>&1

Dataset URL: https://www.kaggle.com/datasets/ayushspai/sportsmot
License(s): MIT


In [6]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, ColorJitter

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from ultralytics import YOLO

import shutil

from huggingface_hub import HfApi, Repository, hf_hub_download

import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Dataset

In [7]:
def load_football_sequences(splits_dir):
    football_file = os.path.join(splits_dir, "football.txt")
    with open(football_file, "r") as f:
        football_sequences = f.read().splitlines()
    return set(football_sequences)

def delete_non_football_data(root_dir):
    splits_dir = os.path.join(root_dir, "splits_txt")
    football_sequences = load_football_sequences(splits_dir)

    for split in ["train", "val", "test"]:
        split_dir = os.path.join(root_dir, "dataset", split)
        if not os.path.exists(split_dir):
            continue

        sequences = os.listdir(split_dir)
        for seq in sequences:
            seq_path = os.path.join(split_dir, seq)

            if seq not in football_sequences:
                # print(f"Deleting non-football sequence: {seq_path}")
                shutil.rmtree(seq_path)

    print("Non-football data deletion complete!")

dataset_dir = "/content/SportsMOT/sportsmot_publish"
delete_non_football_data(dataset_dir)

Non-football data deletion complete!


## Object Detection

In [9]:
def convert_to_yolo_format(root_dir, mot_dir, split):
    os.makedirs(os.path.join(root_dir, "dataset", "images", split), exist_ok=True)
    os.makedirs(os.path.join(root_dir, "dataset", "labels", split), exist_ok=True)

    splits_dir = os.path.join(mot_dir, "splits_txt")
    football_sequences = load_football_sequences(splits_dir)

    for seq in football_sequences:
        seq_path = os.path.join(mot_dir, "dataset", split, seq)
        gt_file = os.path.join(seq_path, "gt", "gt.txt")

        if not os.path.exists(gt_file):
            continue

        df = pd.read_csv(gt_file, header=None)
        df.columns = [
            "frame_id", "object_id", "x", "y", "w", "h",
            "conf", "class_id", "visibility"
        ]

        for frame_id, group in df.groupby("frame_id"):
            img_path = os.path.join(seq_path, "img1", f"{frame_id:06d}.jpg")
            if not os.path.exists(img_path):
                continue

            img = cv2.imread(img_path)
            img_name = f"{seq}_{frame_id:06d}.jpg"
            cv2.imwrite(os.path.join(root_dir, "dataset", "images", split, img_name), img)

            label_file = os.path.join(root_dir, "dataset", "labels", split, f"{seq}_{frame_id:06d}.txt")
            with open(label_file, "w") as f:
                for _, row in group.iterrows():
                    x_center = (row["x"] + row["w"] / 2) / img.shape[1]
                    y_center = (row["y"] + row["h"] / 2) / img.shape[0]
                    width = row["w"] / img.shape[1]
                    height = row["h"] / img.shape[0]
                    f.write(f"{int(row['class_id'])} {x_center} {y_center} {width} {height}\n")

convert_to_yolo_format("/content", "/content/SportsMOT/sportsmot_publish", "train")
convert_to_yolo_format("/content", "/content/SportsMOT/sportsmot_publish", "val")
convert_to_yolo_format("/content", "/content/SportsMOT/sportsmot_publish", "test")

In [10]:
def create_yaml_file(dataset_dir, class_names):
    yaml_content = f"""
path: {dataset_dir}
train: images/train
val: images/val
test: images/test

nc: {len(class_names)}
names: {class_names}
"""
    yaml_path = os.path.join(dataset_dir, "dataset.yaml")
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    print(f"YAML file created at: {yaml_path}")

dataset_dir = "/content/dataset"
class_names = ['player', 'referee', 'ball', 'other']

create_yaml_file(dataset_dir, class_names)

YAML file created at: /content/dataset/dataset.yaml


In [ ]:
model = YOLO("yolov8n.pt")

results = model.train(
    data="/content/dataset/dataset.yaml",
    epochs=50,
    imgsz=1280,
    batch=16,
    name="yolov8_football",
    verbose=False
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 359MB/s]


Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

100%|██████████| 755k/755k [00:00<00:00, 98.9MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 353MB/s]


...

      24/50      3.77G     0.6193     0.3128     0.8125        189        736: 100%|██████████| 695/695 [05:21<00:00,  2.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 284/284 [01:55<00:00,  2.45it/s]


                   all       9058     117024      0.929      0.934      0.942      0.783

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      25/50      3.66G     0.6172     0.3113     0.8118        182        736: 100%|██████████| 695/695 [05:22<00:00,  2.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 284/284 [01:57<00:00,  2.41it/s]


                   all       9058     117024      0.925      0.941      0.943      0.784

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      26/50      3.78G     0.6132     0.3094     0.8105        393        736:  36%|███▌      | 249/695 [01:51<03:00,  2.47it/s]

In [ ]:
def plot_logs(results_path):
    logs = pd.read_csv(results_path)

    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["train/box_loss"], label="Train Box Loss")
    plt.plot(logs["epoch"], logs["val/box_loss"], label="Validation Box Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["metrics/mAP_0.5"], label="mAP@0.5")
    plt.plot(logs["epoch"], logs["metrics/mAP_0.5:0.95"], label="mAP@0.5:0.95")
    plt.xlabel("Epoch")
    plt.ylabel("mAP")
    plt.title("Mean Average Precision")
    plt.legend()
    plt.show()

results_path = "/content/runs/detect/yolov8_football/results.csv"
plot_logs(results_path)

In [ ]:
model = YOLO("/content/runs/detect/yolov8_football/weights/best.pt")

results = model("/content/dataset/images/test/seq101_000001.jpg") # TODO

for result in results:
    result.show()
    result.save("output.jpg")

In [ ]:
metrics = model.val()

print(f"mAP@0.5: {metrics.box.map}")
print(f"mAP@0.5:0.95: {metrics.box.map_50_95}")

## Saving to Hugging Face


In [ ]:
!huggingface-cli login

In [ ]:
model_path = "/content/runs/detect/yolov8_football/weights/best.pt"
repo_id = "ParsaGh/yolov8_football"
repo_name = "yolov8_football"

model = YOLO(model_path)

repo = Repository(l
    ocal_dir=yolov8_football,
    clone_from=repo_id,
    use_auth_token=True
)
os.system(f"cp {model_path} {repo_name}/")
repo.push_to_hub(commit_message="Upload YOLOv8-Football model")

## Loading From Hugging Face

In [ ]:
model_path = hf_hub_download(
    repo_id=repo_id,
    filename="best.pt",
    use_auth_token=True,
)

model = YOLO(model_path)

# results = model("path_to_test_image.jpg") #TODO
# results.show()

## Single Object Tracking

In [ ]:
class SingleObjectTracker:
    def __init__(self, yolo_weights='best.pt'):
        self.yolo = YOLO(yolo_weights)
        self.tracker = cv2.TrackerCSRT_create()
        self.target_class = 0
        self.last_valid_bbox = None
        self.frames_since_last_detection = 0

    def track(self, frame):
        success, bbox = self.tracker.update(frame)

        if not success or self._needs_reinitialization(bbox):
            print("Attempting recovery...")
            success, bbox = self._reinitialize_tracker(frame)

        if success:
            self.last_valid_bbox = bbox
            self.frames_since_last_detection = 0
        else:
            self.frames_since_last_detection += 1

        return success, bbox

    def _needs_reinitialization(self, bbox):
        if bbox[2] <= 0 or bbox[3] <= 0:
            return True
        if self.last_valid_bbox:
            # Check position change > 50% of frame size
            dx = abs(bbox[0] - self.last_valid_bbox[0])/1280
            dy = abs(bbox[1] - self.last_valid_bbox[1])/720
            return dx > 0.5 or dy > 0.5
        return False

    def _reinitialize_tracker(self, frame):
        # Search in last known position area
        if self.last_valid_bbox:
            x, y, w, h = self.last_valid_bbox
            search_region = frame[int(y):int(y+h), int(x):int(x+w)]
            results = self.yolo(search_region, verbose=False)[0]
        else:
            results = self.yolo(frame, verbose=False)[0]

        # Filter and select best detection
        athlete_boxes = []
        for box, cls, conf in zip(results.boxes.xywh, results.boxes.cls, results.boxes.conf):
            if cls == self.target_class and conf > 0.5:
                athlete_boxes.append((conf, box.cpu().numpy()))

        if athlete_boxes:
            # Select closest to last position or highest confidence
            best_conf, best_box = max(athlete_boxes, key=lambda x: x[0])
            x_center, y_center, w, h = best_box
            x = x_center - w/2 + (self.last_valid_bbox[0] if self.last_valid_bbox else 0)
            y = y_center - h/2 + (self.last_valid_bbox[1] if self.last_valid_bbox else 0)
            new_bbox = (x, y, w, h)
            self.tracker = cv2.TrackerCSRT_create()
            self.tracker.init(frame, new_bbox)
            return True, new_bbox

        return False, None

def test_on_sportsmot_frames(data_dir, yolo_model, output_dir="results"):
    tracker = SingleObjectTracker(yolo_model)

    os.makedirs(output_dir, exist_ok=True)

    frames_dir = os.path.join(data_dir, "test", "images")
    frame_paths = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir)])

    frame = cv2.imread(frame_paths[0])
    if frame is None:
        raise ValueError("No frames found in test directory")

    height, width = frame.shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(os.path.join(output_dir, 'tracking_results.mp4'),
                         fourcc, 25.0, (width, height))

    total_time = 0
    first_frame = True

    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        if frame is None:
            continue

        start_time = time.time()

        if first_frame:
            try:
                bbox = tracker.initialize(frame)
                first_frame = False
                status = "Initialized"
            except ValueError as e:
                print(f"Initialization failed: {e}")
                break
        else:
            success, bbox = tracker.track(frame)
            status = "Tracking" if success else "Lost"

        processing_time = time.time() - start_time
        total_time += processing_time

        if not first_frame and success:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.putText(frame, f"Status: {status}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"FPS: {1/processing_time:.1f}", (10, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        out.write(frame)

    out.release()

    avg_fps = len(frame_paths) / total_time
    print(f"Average FPS: {avg_fps:.1f}")
    print(f"Results saved to: {os.path.abspath(output_dir)}")

if __name__ == "__main__":
    # Example usage
    sportsmot_test_dir = "path/to/sportsmot_dataset/test"  # Update this path
    test_on_sportsmot_frames(sportsmot_test_dir)

### Conditions Where CSRT Performs Well:


Slow Motion: The object moves slowly relative to the frame rate.

Minimal Occlusion: The object is not occluded by other objects.

Consistent Lighting: The lighting conditions remain stable.

Distinct Appearance: The object has a unique appearance compared to the background.

### Conditions Where CSRT Performs Poorly:

Fast Motion: The object moves quickly, causing the tracker to lose it.

Occlusion: The object is occluded by other objects or goes out of the frame.

Appearance Changes: The object's appearance changes significantly (e.g., due to lighting or deformation).

Similar Background: The object blends into the background, making it hard to distinguish.